# Video Processing for BT Lab
Video files to png
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [208]:
#import libraries
import os
import re
import cv2
import json
import pandas as pd
import numpy as np
from timeit import default_timer as timer

In [209]:
user_drive = input("Enter user drive: ")
video_path = f"{user_drive}:/Christian/DI_centre_structured"
repo_dir = os.getcwd()
json_dir = repo_dir + "/records/JSON"

In [210]:
def run_video_to_frame(video_path:str , save_folder:str, frame_frequency: dict, new_fps:int):
    video = cv2.VideoCapture(video_path)
    assert video.isOpened()
    vid_fps = int(video.get(cv2.CAP_PROP_FPS))
    number_of_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    vid_length_floored = number_of_frames // vid_fps

    pick_counter, save_counter, set_counter, true_frames = [0, 0, 0, 0]
    list_of_frames = [frame for frame, frequency in frame_frequency.items()]
    expected_frames = new_fps * vid_length_floored
    success = True
    frame_limit = 100

    if not os.path.isdir(save_folder):
        os.mkdir(save_folder)
    else:
        for files in os.listdir(save_folder):
            os.remove(os.path.join(save_folder, files))
    
    print(f"\nSaving frames to: {save_folder}\nPicking frames: {list_of_frames} per set\nExpected number of frames: {expected_frames} ({new_fps}FPS * {vid_length_floored}s).")
    start_time  = timer()

    while success and set_counter < vid_length_floored:
        success, frame = video.read()

        if not success:
            break

        if pick_counter in list_of_frames:
            frame_name = "frame_%d.png" % true_frames
            frame_path = os.path.join(save_folder, frame_name)
            num_of_save = frame_frequency.get(pick_counter)
            
            for _ in range(num_of_save):
                print(f"Saving frame: {frame_name}")
                # cv2.imwrite(frame_path, frame) 
            
            save_counter += 1

        pick_counter += 1
        true_frames += 1

        if (pick_counter == vid_fps):
            set_counter += 1
            pick_counter = 0

        # for testing
        # if true_frames > frame_limit:
        #     print(f"\nLimit set to {frame_limit} frames for testing purposes.")
        #     break
    
    end_time  = timer()
    time_delta = end_time - start_time
    print(f"\nDone in {time_delta} seconds.")
    video.release()

    if save_counter != expected_frames:
        raise ValueError(f"Expected {expected_frames} frames, but got {save_counter} frames")

    return [set_counter, save_counter, true_frames]

In [211]:
def get_frames_path(local_path: str, level:str) -> str:
    fixed_path = local_path.replace("\\", "/")
    fixed_path_split = fixed_path.split("/")
    video_folder = "/".join(fixed_path_split[:-1])
    video_filename = fixed_path_split[-1].split(".")[0]
    folder_path = video_folder + f"/frames_{video_filename}_{level}"
    return folder_path

In [212]:
def down_sample(old_fps, new_fps):
    down_sampled_list = []
    user_frames = np.arange(0, old_fps, dtype=int)
    step = old_fps / new_fps

    for fps in range(new_fps):
        start = int(fps * step)
        end = int((fps + 1) * step)
        
        if fps == new_fps - 1:
            end = old_fps # ensures that the last element is included
        
        avg = sum(user_frames[start:end]) // (end - start)
        down_sampled_list.append(avg)

    return down_sampled_list

In [213]:
def get_frequency(list_item: list):
    frequency = {}
    for item in list_item:
        frequency[item] = frequency.get(item, 0) + 1
    return frequency

In [214]:
def up_sample(old_fps, new_fps):
    frames_arr = np.arange(0, old_fps, dtype=int)
    frames_interp = np.linspace(0, old_fps - 1, new_fps)
    up_sampled_list = np.interp(frames_interp, frames_arr, frames_arr)
    rounded_up_sampled_list = up_sampled_list.astype(int)

    return rounded_up_sampled_list.tolist()

In [215]:
def convert_video_to_frame(all_patients:dict, level: str, new_fps:int):
    for json_index, patient_info in all_patients.items():
        try:
            video_path = patient_info["local path"]
            old_fps = int(patient_info["old fps"])

            frames_folder = get_frames_path(video_path, level)
            
            if old_fps <= new_fps:
                frames_to_pick = up_sample(old_fps, new_fps)
            else:
                frames_to_pick = down_sample(old_fps, new_fps)
            
            if len(frames_to_pick) != new_fps:
                raise ValueError("Number of frames to pick is not equal to new fps")

            frame_frequency = get_frequency(frames_to_pick)
            set_counter, save_counter, true_frames = run_video_to_frame(video_path, frames_folder, frame_frequency, new_fps)
            print(f"\nSet counter: {set_counter}, save counter: {save_counter}, frame counter: {true_frames}\n\n"+ "-"*50)
        except Exception as e:
            print(f'''{type(e)}: {e} for video {patient_info["filename"]}''')

In [216]:
def load_json(json_dir:str, filename:str) -> dict:
    full_path = json_dir + "/" + filename

    with open(full_path, "r") as json_data:
        data = json.load(json_data)

    return(data)

## Video to frames (stage 3.1)

Based on the desired fps, turn videos into frames

In [217]:
""" local vals"""

rgb_fps = {
    "lower_bound": 10,
    "upper_bound": 20
}

thermal_fps = {
    "lower_bound": 5,
    "upper_bound": 10
}

In [218]:
""" load JSON files """

metadata_rgb = load_json(json_dir, "rgb_test.json")
metadata_thermal = load_json(json_dir, "thermal_test.json")

In [219]:
""" convert video to png (rgb) """

for level, new_fps in rgb_fps.items():
    print(f"\nAdjusting FPS to {new_fps}\n" + "="*50)
    convert_video_to_frame(metadata_rgb, level, new_fps)


Adjusting FPS to 10

Saving frames to: D:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Hold Breath/frames_Arun2_lower_bound
Picking frames: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27] per set
Expected number of frames: 740 (10FPS * 74s).
Saving frame: frame_0.png
Saving frame: frame_3.png
Saving frame: frame_6.png
Saving frame: frame_9.png
Saving frame: frame_12.png
Saving frame: frame_15.png
Saving frame: frame_18.png
Saving frame: frame_21.png
Saving frame: frame_24.png
Saving frame: frame_27.png
Saving frame: frame_29.png
Saving frame: frame_32.png
Saving frame: frame_35.png
Saving frame: frame_38.png
Saving frame: frame_41.png
Saving frame: frame_44.png
Saving frame: frame_47.png
Saving frame: frame_50.png
Saving frame: frame_53.png
Saving frame: frame_56.png
Saving frame: frame_58.png
Saving frame: frame_61.png
Saving frame: frame_64.png
Saving frame: frame_67.png
Saving frame: frame_70.png
Saving frame: frame_73.png
Saving frame: frame_76.png
Saving 